## Algoritmo Polymarket

In [1]:
import requests
import json
import asyncio
import websockets
import datetime
from datetime import datetime,timedelta
import re

# ==============================
# 🔹 VARIABLE GLOBAL: Nombre del mercado a buscar
MERCADO = "Uefa Champions League Winner"
SLUG_MERCADO = re.sub(r"\s+", "-", re.sub(r"[^\w\s]", "", MERCADO.lower())).strip("-")

# ==============================
# URL API REST
API_URL = "https://gamma-api.polymarket.com/events?slug=$SLUG_MERCADO"
WS_URL = "wss://ws-subscriptions-clob.polymarket.com/ws/market"


## Buscar mercado y sus características

In [3]:
def formato_legible(fecha_str):
    if fecha_str:
        return datetime.fromisoformat(fecha_str.replace("Z", "+00:00")).strftime("%d/%m/%Y %H:%M:%S")
    return "N/A"

def elegir_submarket(slug_evento):
    """
    Busca un evento por slug y permite elegir un sub-market/candidato.
    Devuelve los token_ids del sub-market elegido y muestra características clave.
    """
    url = f"https://gamma-api.polymarket.com/events?slug={slug_evento}"
    r = requests.get(url)
    
    if r.status_code != 200:
        print(f"❌ Error al acceder a la API ({r.status_code})")
        return []

    response = r.json()
    if not response:
        print(f"❌ No se encontró ningún evento con slug '{slug_evento}'")
        return []

    event = response[0]
    print(f"✅ Evento encontrado: {event.get('title', 'N/A')}")
    print(f"ID del evento: {event.get('id', 'N/A')}")
    print(f"Descripción: {event.get('description', 'N/A')}")
    print(f"Fecha de inicio: {formato_legible(event.get('startDate'))}")
    print(f"Fecha de cierre: {formato_legible(event.get('endDate'))}\n")

    markets = event.get("markets", [])
    if not markets:
        print("❌ No hay sub-markets disponibles")
        return []

    print("Sub-markets disponibles (por candidato):")
    for i, m in enumerate(markets):
        print(f"  {i}: {m.get('question', 'N/A')}")

    # Elegir sub-market
    while True:
        choice = input(f"\nElige el índice del sub-market que quieres escuchar (0-{len(markets)-1}): ")
        if choice.isdigit() and 0 <= int(choice) < len(markets):
            market = markets[int(choice)]
            tokens = json.loads(market.get("clobTokenIds", "[]"))
            outcomes = json.loads(market.get("outcomes", "[]"))

            print(f"\n✔️ Has elegido: {market.get('question')}")
            print(f"Outcomes: {outcomes}")
            print(f"Token IDs: {tokens}\n")

            # Mostrar características principales ( Estas son solo un ejemplo)
            print(market)
            print("=== Características principales del sub-market ===")
            for k in ["liquidity", "volume", "outcomePrices", "volume24hrClob", "orderPriceMinTickSize", 
                      "orderMinSize", "lastTradePrice", "bestBid", "bestAsk", "spread", "competitive"]:
                print(f"{k}: {market.get(k, 'N/A')}")

            return tokens

        print("❌ Entrada inválida, intenta de nuevo.")

## Conectarse a Polymarket


* Nos conectamos a Polymarket y accedemos en tiempo real al CLOB con ambas respuestas 

In [ ]:
last_time_pong = datetime.now()

async def main():
    global last_time_pong

    # 🔹 Elección del sub-market ANTES del while loop
    tokens = elegir_submarket(SLUG_MERCADO)
    if not tokens:
        return

    async with websockets.connect(WS_URL) as websocket:
        # Suscribirse a los tokens elegidos
        await websocket.send(json.dumps({
            "assets_ids": tokens,
            "type": "market"
        }))

        while True:
            msg = await websocket.recv()

            if msg != "PONG":
                last_time_pong = datetime.now()

            try:
                data = json.loads(msg)
            except json.JSONDecodeError:
                continue

            events = data if isinstance(data, list) else [data]

            for ev in events:
                if ev.get("event_type") == "book":
                    asset = ev.get("asset_id")
                    bids = ev.get("bids", [])
                    asks = ev.get("asks", [])

                    # 🔹 Best Bid = máximo de los bids, Best Ask = mínimo de los asks
                    best_bid = max([float(b["price"]) for b in bids]) if bids else 0
                    best_ask = min([float(a["price"]) for a in asks]) if asks else 0
                    mid_price = round((best_bid + best_ask) / 2, 4) if bids and asks else 0
                    spread = round(abs(best_ask - best_bid), 4) if bids and asks else 0

                    print(f"\n=== Asset ID: {asset} ===")
                    print(f"Best Bid: {best_bid} | Best Ask: {best_ask} | Mid Price: {mid_price} | Spread: {spread}")
                    print("Top Bids:")
                    for b in sorted(bids, key=lambda x: float(x["price"]), reverse=True)[:5]:
                        print(f"  Price: {b['price']}, Size: {b['size']}")
                    print("Top Asks:")
                    for a in sorted(asks, key=lambda x: float(x["price"]))[:5]:
                        print(f"  Price: {a['price']}, Size: {a['size']}")

            # Mantener la conexión viva
            if last_time_pong + timedelta(seconds=10) < datetime.now():
                await websocket.send("PING")


# 🔹 SOLO para Jupyter/Notebook
await main()

✅ Evento encontrado: UEFA Champions League Winner 
ID del evento: 33506
Descripción: This is a polymarket to predict which club will win the 2025–26 UEFA Champions League (soccer). 
Fecha de inicio: 28/07/2025 16:48:06
Fecha de cierre: 31/05/2026 00:00:00

Sub-markets disponibles (por candidato):
  0: Will Real Madrid win the 2025–26 Champions League?
  1: Will Arsenal win the 2025–26 Champions League?
  2: Will Nice win the 2025–26 Champions League?
  3: Will Man City win the 2025–26 Champions League?
  4: Will Dortmund win the 2025–26 Champions League?
  5: Will Slavia Pragu win the 2025–26 Champions League?
  6: Will Team H win the 2025–26 Champions League?
  7: Will Bayern Munich win the 2025–26 Champions League?
  8: Will Newcastle win the 2025–26 Champions League?
  9: Will Galatasaray win the 2025–26 Champions League?
  10: Will Atletico Madrid win the 2025–26 Champions League?
  11: Will Napoli win the 2025–26 Champions League?
  12: Will Tottenham win the 2025–26 Champions Lea